In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/archive.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [10]:
import tensorflow as tf
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, LeakyReLU, BatchNormalization, Dropout, Dense, InputLayer, Flatten
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam

In [15]:
gen = ImageDataGenerator(rescale=1./255,validation_split = 0.2,zoom_range=(0.99,0.99),dtype=None)

In [16]:
train = gen.flow_from_directory("/tmp/Brain Tumor Data Set/Brain Tumor Data Set",
                               target_size = (150,150),
                               batch_size = 256,
                               class_mode = "binary",
                               color_mode = "rgb",
                               shuffle = True,
                               seed = 123,
                               subset = "training")

Found 3681 images belonging to 2 classes.


In [17]:
val = gen.flow_from_directory("/tmp/Brain Tumor Data Set/Brain Tumor Data Set",
                               target_size = (150,150),
                               batch_size = 8,
                               class_mode = "binary",
                               color_mode = "rgb",
                               shuffle = True,
                               seed = 123,
                               subset = "validation")

Found 919 images belonging to 2 classes.


In [18]:
model = keras.Sequential()
model.add(InputLayer(input_shape=(150,150,3)))
model.add(Conv2D(filters=32,kernel_size=3, activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=3, activation="relu", padding="same"))
model.add(MaxPool2D())


model.add(Flatten())


model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
model.add(Dense(64, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
model.add(Dense(1, activation="sigmoid"))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 87616)             0         
                                                                 
 dense_3 (Dense)             (None, 128)              

In [20]:
model.compile(optimizer=Adam(0.001),loss = BinaryCrossentropy(),metrics=['accuracy'])

In [21]:
from keras import utils, callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min",
                                        patience=5, restore_best_weights = True)

In [22]:
history = model.fit(train,verbose=1,callbacks = [earlystopping],epochs=15,validation_data=(val))

Epoch 1/15
15/15 [==============================] - 41s 2s/step - loss: 0.6219 - accuracy: 0.7004 - val_loss: 0.7390 - val_accuracy: 0.7410
Epoch 2/15
15/15 [==============================] - 29s 2s/step - loss: 0.3709 - accuracy: 0.8378 - val_loss: 0.5500 - val_accuracy: 0.7138
Epoch 3/15
15/15 [==============================] - 28s 2s/step - loss: 0.2107 - accuracy: 0.9239 - val_loss: 0.3161 - val_accuracy: 0.8792
Epoch 4/15
15/15 [==============================] - 35s 2s/step - loss: 0.1344 - accuracy: 0.9568 - val_loss: 0.2535 - val_accuracy: 0.9108
Epoch 5/15
15/15 [==============================] - 28s 2s/step - loss: 0.0836 - accuracy: 0.9761 - val_loss: 0.1959 - val_accuracy: 0.9489
Epoch 6/15
15/15 [==============================] - 28s 2s/step - loss: 0.0534 - accuracy: 0.9891 - val_loss: 0.2029 - val_accuracy: 0.9695
Epoch 7/15
15/15 [==============================] - 28s 2s/step - loss: 0.0342 - accuracy: 0.9951 - val_loss: 0.2350 - val_accuracy: 0.9260
Epoch 8/15
15/15 [==

In [23]:
model.save('brain_tumor.keras')

In [24]:
trained_model = keras.models.load_model('/content/brain_tumor.keras')